In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import date
import holidays

### Import Dataset

In [15]:
cliente    = '09517617000153_3.csv'
path_miner = '../../../data/miner/'
path_res   = '../../../data/res/'

id_produto = '80354'
name_end = 'df_qtd_rf_' + id_produto

data = cliente

In [16]:
df   = pd.read_csv(path_miner + data, encoding="utf-8", dtype=str)

### Functions

In [17]:
def isWeekend(n):
    if n <5:
        return 0
    else:
        return 1

In [18]:
def isStartMes(n):
    if ( n > 30):
        return 1
    elif (n < 10):
        return 1
    else:
        return 0

In [19]:
def isHoliday(date):
    br_holidays = holidays.BR()
    if ((date in br_holidays)):
        return 1
    else:
        return 0

In [20]:
def ref(v1,v2, c):
    if (v1 > v2):
        div = (v1 - v2)/v1
        if (div > c):
            return v1
        else:
            return v2
    else:
        div = (v2 - v1)/v2
        if (div>c):
            return v1
        else:
            return v2

### Processing

In [21]:
df[["data"]]                 = df[["data"]].apply(pd.to_datetime)
df[["valor"]]                = df[["valor"]].apply(pd.to_numeric)
df[["quantidade"]]           = df[["quantidade"]].apply(pd.to_numeric)
df[["valorCancelado"]]       = df[["valorCancelado"]].apply(pd.to_numeric)
df[["quantidadeCancelada"]]  = df[["quantidadeCancelada"]].apply(pd.to_numeric)

In [22]:
df_prod    = df[df['produtoId'] == id_produto]

In [23]:
df_prod = df_prod.sort_values(by=['data'],ascending=True)

In [24]:
df_prod["valor_unit"]  =  round(df_prod["valor"]/df["quantidade"],2)
df_prod['ano']         =  pd.DatetimeIndex(df_prod['data']).year
df_prod['mes']         =  pd.DatetimeIndex(df_prod['data']).month
df_prod['dia_semana']  =  pd.DatetimeIndex(df_prod['data']).dayofweek
df_prod['dia']         =  pd.DatetimeIndex(df_prod['data']).day
df_prod['fim_semana']  =  df_prod['dia_semana'].apply(lambda x: isWeekend((x)))
df_prod['inicio_mes']  =  df_prod['dia'].apply(lambda x: isStartMes((x)))
df_prod['feriado']     =  df_prod['data'].apply(lambda x: isHoliday((x)))

In [25]:
columns = ['data', 'produtoId', 'descr', 'valor', 'valorCancelado', 'quantidadeCancelada', 'quantidadeDocumentos', 'quantidadeValorZero']
df_prod.drop(columns, inplace=True, axis=1)

In [26]:
preco_ref = df_prod.mode(axis=0)['valor_unit'][0]
var_preco = 0.05

df_prod["valor_unit"]  = df_prod['valor_unit'].apply(lambda x: ref(x, preco_ref, var_preco) )

In [27]:
df_prod.index = range(0,len(df_prod))
df_prod = df_prod[['valor_unit', 'mes', 'dia_semana', 'dia', 'inicio_mes', 'fim_semana', 'feriado', 'quantidade']]

In [28]:
df_prod_res = df_prod

In [29]:
df_prod_res.to_csv(path_res +  name_end + '.csv')